In [ ]:
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium import webdriver
from urllib import request
from tabulate import tabulate
import pandas as pd
import requests
import luigi
import time
import json
import csv 
import re
import os

In [2]:
class ExtractTokpedExsportData(luigi.Task):
    def requires(self):
        pass # Tidak ada task yang diperlukan
    
    def output(self):
        return luigi.LocalTarget('extract-raw-data/exsport_tokped_raw.csv') # MTempat penyimpanan data yang diekstrak


    def run(self):
        base_url = "https://www.tokopedia.com/exsportstore/product/page/{}" # URL dasar untuk mengambil data produk exsport dari Tokopedia

        # Mengatur opsi untuk webdriver Chrome
        options = webdriver.ChromeOptions()
        options.add_argument('--disable-blink-features=AutomationControlled') # Menonaktifkan fitur otomatisasi
        options.add_experimental_option('useAutomationExtension', False) # Menonaktifkan ekstensi otomatisasi
        options.add_experimental_option("excludeSwitches", ["enable-automation"]) # Mengecualikan switch otomatisasi
        driver = webdriver.Chrome(options=options) # Membuat instance dari webdriver Chrome

        product_data = [] # List untuk menyimpan data produk

        try:
            for page in range(1, 12): # Mengambil data dari halaman 1 hingga 11
                url = base_url.format(page) # Membuat URL untuk halaman saat ini
                driver.get(url) # Mengakses URL

                # Menunggu hingga elemen body muncul
                WebDriverWait(driver, 15).until(
                    EC.presence_of_element_located((By.TAG_NAME, 'body'))
                )

                # Menggulir halaman untuk memuat lebih banyak produk
                for _ in range(5):
                    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);") # Menggulir ke bawah
                    time.sleep(2) # Menunggu 2 detik
                    driver.execute_script("window.scrollTo(0, document.body.scrollHeight / 2);") # Menggulir ke atas
                    time.sleep(2) # Menunggu 2 detik

                # Mengambil elemen produk
                product_containers = driver.find_elements(By.CSS_SELECTOR, "[data-testid='divProductWrapper']")

                for container in product_containers: # Iterasi setiap elemen produk
                    try:
                        name = container.find_element(By.CSS_SELECTOR, "[data-testid='linkProductName']").text # Mengambil nama produk
                    except:
                        name = None # Jika gagal, set nama menjadi None
                    
                    # Mengambil link produk dari elemen
                    try:
                        link = container.find_element(By.CSS_SELECTOR, "a.pcv3__info-content").get_attribute('href') # Mencari elemen link produk dan mengambil atribut 'href'
                    except:
                        link = None # Jika gagal, set link menjadi None

                    # Mengambil harga jual produk dari elemen
                    try:
                        price_sale_elem = container.find_element(By.CSS_SELECTOR, "[data-testid='linkProductPrice']") # Mencari elemen harga jual produk
                        price_sale = price_sale_elem.text if price_sale_elem else None # Mengambil teks dari elemen harga jual produk
                    except:
                        price_sale = None # Jika gagal, set harga jual menjadi None

                    # Mengambil harga asli produk dari elemen
                    try:
                        price_elem = container.find_element(By.CSS_SELECTOR, "[data-testid='lblProductSlashPrice']") # Mencari elemen harga asli produk
                        price = price_elem.text if price_elem else None # Mengambil teks dari elemen harga asli produk
                    except:
                        price = None # Jika gagal, set harga asli menjadi None

                    try:
                        discount_elem = container.find_element(By.CSS_SELECTOR, "[data-testid='lblProductDiscount']") # Mencari elemen diskon produk
                        discount = discount_elem.text if discount_elem else None # Mengambil teks dari elemen diskon produk
                    except:
                        discount = None # Jika gagal, set diskon menjadi None

                    # Mengambil rating produk dari elemen
                    try:
                        rating_elem = container.find_element(By.CSS_SELECTOR, ".prd_rating-average-text") # Mencari elemen rating produk
                        rating = rating_elem.text if rating_elem else None # Mengambil teks dari elemen rating produk
                    except:
                        rating = None # Jika gagal, set rating menjadi None
                    
                    # Mengambil jumlah produk yang terjual dari elemen
                    try:
                        sold_elem = container.find_element(By.CSS_SELECTOR, ".prd_label-integrity") # Mencari elemen jumlah produk yang terjual
                        sold = sold_elem.text if sold_elem else None # Mengambil teks dari elemen jumlah produk yang terjual
                    except:
                        sold = None # Jika gagal, set jumlah produk yang terjual menjadi None

                    # Mengambil link gambar produk dari elemen
                    try:
                        image_elem = container.find_element(By.CSS_SELECTOR, ".css-1q90pod") # Mencari elemen gambar produk
                        image = image_elem.get_attribute('src') if image_elem else None # Mengambil atribut 'src' dari elemen gambar produk
                    except:
                        image = None # Jika gagal, set link gambar menjadi None

                    # Menambahkan data produk ke dalam list product_data
                    product_data.append({
                        'name_product': name, # Nama produk
                        'product_link': link, # Link produk
                        'price_sale': price_sale, # Harga jual
                        'price_original': price, # Harga asli
                        'discount': discount, # Diskon
                        'sold': sold, # Jumlah produk yang terjual
                        'rating': rating, # Rating produk
                        'image_link': image # Link gambar produk
                    })

            # Mengonversi list product_data ke dalam DataFrame
            exsport_tokped_df = pd.DataFrame(product_data)

            # Menyimpan DataFrame ke dalam file CSV
            exsport_tokped_df.to_csv(self.output().path, index=False)

        except Exception as e:
            print(f"Terjadi kesalahan: {e}") # Menampilkan pesan kesalahan jika terjadi kesalahan
        
        finally:
            driver.quit() # Menutup browser

In [14]:
luigi.build([ExtractTokpedExsportData()], local_scheduler=True) # Menjalankan task ExtractTokpedExsportData

DEBUG: Checking if ExtractTokpedExsportData() is complete
INFO: Informed scheduler that task   ExtractTokpedExsportData__99914b932b   has status   PENDING
INFO: Done scheduling tasks
INFO: Running Worker with 1 processes
DEBUG: Asking scheduler for work...
DEBUG: Pending tasks: 1
INFO: [pid 26588] Worker Worker(salt=3377661383, workers=1, host=zueible, username=LENOVO, pid=26588) running   ExtractTokpedExsportData()
INFO: [pid 26588] Worker Worker(salt=3377661383, workers=1, host=zueible, username=LENOVO, pid=26588) done      ExtractTokpedExsportData()
DEBUG: 1 running tasks, waiting for next task to finish
INFO: Informed scheduler that task   ExtractTokpedExsportData__99914b932b   has status   DONE
DEBUG: Asking scheduler for work...
DEBUG: Done
DEBUG: There are no more tasks to run at this time
INFO: Worker Worker(salt=3377661383, workers=1, host=zueible, username=LENOVO, pid=26588) was stopped. Shutting down Keep-Alive thread
INFO: 
===== Luigi Execution Summary =====

Scheduled 1 t

True

In [ ]:
class ExtractTokpedStockExsportData(luigi.Task):
    def requires(self):
        return ExtractTokpedExsportData() # Task yang diperlukan
    
    def output(self):
        return luigi.LocalTarget('extract-raw-data/exsport_stock_tokped_raw.csv') # Tempat penyimpanan data yang diekstrak


    def run(self):
        extract_data = pd.read_csv(self.input().path) # Membaca file CSV yang diekstrak sebelumnya
        pages = extract_data['product_link'].tolist() # Mengambil kolom product_link dan mengonversinya ke dalam list

        # Mengatur opsi untuk webdriver Chrome
        options = Options()
        options.add_argument("--headless")
        options.add_argument("--disable-gpu")
        options.add_argument("--window-size=1920x1080")
        driver = webdriver.Chrome(options=options) # Membuat instance dari webdriver Chrome

        product_data = [] # List untuk menyimpan data produk

        try:
            for page in pages: # Mengambil data dari halaman 1 hingga 11
                url = page # Membuat URL untuk halaman saat ini
                driver.get(url) # Mengakses URL

                # Menunggu hingga elemen body muncul
                WebDriverWait(driver, 15).until(
                    EC.presence_of_element_located((By.TAG_NAME, 'body'))
                )

                # Menggulir halaman untuk memuat lebih banyak produk
                for _ in range(5):
                    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);") # Menggulir ke bawah
                    time.sleep(2) # Menunggu 2 detik
                    driver.execute_script("window.scrollTo(0, document.body.scrollHeight / 2);") # Menggulir ke atas
                    time.sleep(2) # Menunggu 2 detik

                # Mengambil elemen produk
                product_containers = driver.find_elements(By.CSS_SELECTOR, "[class='css-856ghu']")

                for container in product_containers: # Iterasi setiap elemen produk
                    try:
                        name = container.find_element(By.CSS_SELECTOR, "[data-testid='lblPDPDetailProductName']").text # Mengambil nama produk
                    except:
                        name = None # Jika gagal, set nama menjadi None

                    # Mengambil stock jual produk dari elemen
                    try:
                        stock_elem = container.find_element(By.CSS_SELECTOR, "[data-testid='stock-label']")  # Mencari elemen stok
                        stock_text = stock_elem.text.strip()  # Mengambil teks dan menghapus spasi ekstra
                        
                        # Mengambil angka stok setelah "Stok Total: "
                        stock = stock_text.split(":")[-1].strip()  
                    except:
                        stock = None  # Jika gagal, set stok menjadi None

                    # Mengambil kategori produk dari elemen
                    try:
                        # Mencari elemen etalase berdasarkan class
                        etalase_elem = container.find_element(By.CSS_SELECTOR, "li.css-1i6xy22 a b")
                        etalase = etalase_elem.text.strip()  # Mengambil teks dari elemen <b>
                    except:
                        etalase = None  # Jika gagal, set etalase menjadi None
                    
                    try:
                        description_elem = container.find_element(By.CSS_SELECTOR, "[data-testid='lblPDPDescriptionProduk']")
                        description_html = description_elem.get_attribute('innerHTML')  # Mengambil HTML untuk memproses <br>
                        
                        # Memproses <br> agar dipisahkan dengan pemisah tertentu
                        description_text = description_html.replace("<br>", "|")
                        
                        # Ambil bagian setelah <br> ketiga
                        parts = description_text.split("|")
                        description = parts[3].strip() if len(parts) > 3 else None

                    except:
                        description = None

                    # Menambahkan data produk ke dalam list product_data
                    product_data.append({
                        'name_product': name,
                        'stock': stock,
                        'kategori': etalase,
                        'description': description,  # Tambahkan seluruh deskripsi
                    })


            # Mengonversi list product_data ke dalam DataFrame
            exsport_stock_tokped_df = pd.DataFrame(product_data)

            # Menyimpan DataFrame ke dalam file CSV
            exsport_stock_tokped_df.to_csv(self.output().path, index=False)

        except Exception as e:
            print(f"Terjadi kesalahan: {e}") # Menampilkan pesan kesalahan jika terjadi kesalahan
        
        finally:
            driver.quit() # Menutup browser

In [4]:
luigi.build([ExtractTokpedStockExsportData()], local_scheduler=True) # Menjalankan task ExtractTokpedStockExsportData

DEBUG: Checking if ExtractTokpedStockExsportData() is complete
DEBUG: Checking if ExtractTokpedExsportData() is complete
INFO: Informed scheduler that task   ExtractTokpedStockExsportData__99914b932b   has status   PENDING
INFO: Informed scheduler that task   ExtractTokpedExsportData__99914b932b   has status   DONE
INFO: Done scheduling tasks
INFO: Running Worker with 1 processes
DEBUG: Asking scheduler for work...
DEBUG: Pending tasks: 1
INFO: [pid 16820] Worker Worker(salt=2618818627, workers=1, host=zueible, username=LENOVO, pid=16820) running   ExtractTokpedStockExsportData()


Terjadi kesalahan: Message: timeout: Timed out receiving message from renderer: -0.008
  (Session info: chrome=134.0.6998.89)
Stacktrace:
	GetHandleVerifier [0x00007FF7E933FE65+26629]
	(No symbol) [0x00007FF7E92A6030]
	(No symbol) [0x00007FF7E913931A]
	(No symbol) [0x00007FF7E912662C]
	(No symbol) [0x00007FF7E912631A]
	(No symbol) [0x00007FF7E9123E9F]
	(No symbol) [0x00007FF7E912496F]
	(No symbol) [0x00007FF7E913366E]
	(No symbol) [0x00007FF7E9149823]
	(No symbol) [0x00007FF7E915094A]
	(No symbol) [0x00007FF7E91250DD]
	(No symbol) [0x00007FF7E9149555]
	(No symbol) [0x00007FF7E91E004A]
	(No symbol) [0x00007FF7E91B7883]
	(No symbol) [0x00007FF7E9180550]
	(No symbol) [0x00007FF7E9181803]
	GetHandleVerifier [0x00007FF7E96972DD+3529853]
	GetHandleVerifier [0x00007FF7E96ADA42+3621858]
	GetHandleVerifier [0x00007FF7E96A24F3+3575443]
	GetHandleVerifier [0x00007FF7E940B79A+860474]
	(No symbol) [0x00007FF7E92B08AF]
	(No symbol) [0x00007FF7E92ACBE4]
	(No symbol) [0x00007FF7E92ACD86]
	(No symbol) 

INFO: [pid 16820] Worker Worker(salt=2618818627, workers=1, host=zueible, username=LENOVO, pid=16820) done      ExtractTokpedStockExsportData()
DEBUG: 1 running tasks, waiting for next task to finish
INFO: Informed scheduler that task   ExtractTokpedStockExsportData__99914b932b   has status   DONE
DEBUG: Asking scheduler for work...
DEBUG: Done
DEBUG: There are no more tasks to run at this time
INFO: Worker Worker(salt=2618818627, workers=1, host=zueible, username=LENOVO, pid=16820) was stopped. Shutting down Keep-Alive thread
INFO: 
===== Luigi Execution Summary =====

Scheduled 2 tasks of which:
* 1 complete ones were encountered:
    - 1 ExtractTokpedExsportData()
* 1 ran successfully:
    - 1 ExtractTokpedStockExsportData()

This progress looks :) because there were no failed tasks or missing dependencies

===== Luigi Execution Summary =====



True

In [6]:
class ValidateData(luigi.Task):
    def requires(self):
        return [ExtractTokpedExsportData(), ExtractTokpedStockExsportData()] # Task yang diperlukan
    
    def output(self):
        return luigi.LocalTarget('validate-raw-data/validate_data.txt')

    def run(self):
        # Buat direktori jika belum ada
        os.makedirs(os.path.dirname(self.output().path), exist_ok=True)
        
        raw_exsport_data = pd.read_csv(self.input()[0].path) # Membaca file CSV yang diekstrak sebelumnya
        raw_exsport_stock_data = pd.read_csv(self.input()[1].path) # Membaca file CSV yang diekstrak sebelumnya

        list_df = {
            'raw_exsport_data': raw_exsport_data,
            'raw_exsport_stock_data': raw_exsport_stock_data
        }
        
        with open(self.output().path, 'w', newline='') as f:
            # Check Data Shape
            f.write("==================== Check Data Shape ====================\n\n")
            result_df = []
            for name, df in list_df.items():
                n_columns = df.shape[1]
                n_rows = df.shape[0]
                result_df.append([name, n_columns, n_rows])
                
            headers_shape = ['Dataframe', 'Columns', 'Rows']
            f.write(tabulate(result_df, headers_shape, tablefmt="grid"))
            f.write("\n\n")

            # Check Data Values
            for name, df in list_df.items():
                result_val = []
                for col in df.columns:
                    col_type = df[col].dtype
                    sum_na = round(df[col].isna().sum() * 100 / len(df))
                    sum_dup = round(df.duplicated(keep=False).sum())
                    result_val.append([col, col_type, sum_na, sum_dup])

                headers_val = ['Column Name', 'Data Type', 'Missing Values (%)', 'Duplicate Values (count)']
                f.write(f"Checking Data Values: {name}\n")
                f.write(tabulate(result_val, headers_val, tablefmt="grid"))
                f.write("\n\n")

In [7]:
luigi.build([ValidateData()], local_scheduler=True) # Menjalankan task ValidateData

DEBUG: Checking if ValidateData() is complete
DEBUG: Checking if ExtractTokpedExsportData() is complete
DEBUG: Checking if ExtractTokpedStockExsportData() is complete
INFO: Informed scheduler that task   ValidateData__99914b932b   has status   PENDING
INFO: Informed scheduler that task   ExtractTokpedStockExsportData__99914b932b   has status   DONE
INFO: Informed scheduler that task   ExtractTokpedExsportData__99914b932b   has status   DONE
INFO: Done scheduling tasks
INFO: Running Worker with 1 processes
DEBUG: Asking scheduler for work...
DEBUG: Pending tasks: 1
INFO: [pid 23944] Worker Worker(salt=2009130413, workers=1, host=zueible, username=LENOVO, pid=23944) running   ValidateData()
INFO: [pid 23944] Worker Worker(salt=2009130413, workers=1, host=zueible, username=LENOVO, pid=23944) done      ValidateData()
DEBUG: 1 running tasks, waiting for next task to finish
INFO: Informed scheduler that task   ValidateData__99914b932b   has status   DONE
DEBUG: Asking scheduler for work...
D

True

In [ ]:
class TransformData(luigi.Task):
    def requires(self):
        return [ExtractTokpedExsportData(), ExtractTokpedStockExsportData()] # Task yang diperlukan
    
    def output(self):
        return luigi.LocalTarget('transform-data/data-clean') # Tempat penyimpanan data yang telah di-transform


    def run(self):
        exsport_data = pd.read_csv(self.input()[0].path) # Membaca file CSV yang diekstrak sebelumnya
        exsport_stock_data = pd.read_csv(self.input()[1].path) # Membaca file CSV yang diekstrak sebelumnya
        

        
        
        

In [51]:
df = pd.read_csv('extract-raw-data/exsport_tokped_raw.csv', index_col=False)
df.head()

,name_product,product_link,price_sale,price_original,discount,sold,rating,image_link
0,Exsport All Set Multipurpose Pouch - Dark Green,https://www.tokopedia.com/exsportstore/exsport...,Rp141.550,Rp149.000,5%,NaN,NaN,https://images.tokopedia.net/img/cache/200-squ...
1,Exsport All Set Multipurpose Pouch - Light Brown,https://www.tokopedia.com/exsportstore/exsport...,Rp141.550,Rp149.000,5%,NaN,NaN,https://images.tokopedia.net/img/cache/200-squ...
2,Exsport All Set Multipurpose Pouch - Dark Purple,https://www.tokopedia.com/exsportstore/exsport...,Rp141.550,Rp149.000,5%,NaN,NaN,https://images.tokopedia.net/img/cache/200-squ...
3,Exsport All Set Multipurpose Pouch - Black,https://www.tokopedia.com/exsportstore/exsport...,Rp141.550,Rp149.000,5%,NaN,NaN,https://images.tokopedia.net/img/cache/200-squ...
4,Exsport Basic Half Moon Mini Sling Bag - Blue...,https://www.tokopedia.com/exsportstore/exsport...,Rp170.050,Rp179.000,5%,NaN,NaN,https://images.tokopedia.net/img/cache/200-squ...


In [34]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 802 entries, 0 to 801
Data columns (total 8 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   name_product    802 non-null    object 
 1   product_link    802 non-null    object 
 2   price_sale      802 non-null    object 
 3   price_original  327 non-null    object 
 4   discount        327 non-null    object 
 5   sold            720 non-null    object 
 6   rating          662 non-null    float64
 7   image_link      802 non-null    object 
dtypes: float64(1), object(7)
memory usage: 50.3+ KB


In [52]:
df['color_product'] = df['name_product'].str.split('-').str[-1]
df['name_product'] = df['name_product'].str.split('-').str[0]

In [54]:
df['price_original'] = df['price_original'].fillna(df['price_sale'])

In [55]:
df['price_original'] = df['price_original'].str.replace('Rp', '').str.replace('.', '').str.strip().apply(pd.to_numeric)
df['price_sale'] = df['price_sale'].str.replace('Rp', '').str.replace('.', '').str.strip().apply(pd.to_numeric)

In [57]:
df['discount'] = df['discount'].fillna('No Discount')

In [60]:
df.tail(20)

,name_product,product_link,price_sale,price_original,discount,sold,rating,image_link,color_product
782,Tas Exsport Percy Waist Bag,https://www.tokopedia.com/exsportstore/tas-exs...,139500,139500,No Discount,7 terjual,4.8,https://images.tokopedia.net/img/cache/200-squ...,Cream 3L
783,Tas Exsport Percy Waist Bag,https://www.tokopedia.com/exsportstore/tas-exs...,139500,139500,No Discount,1 terjual,5.0,https://images.tokopedia.net/img/cache/200-squ...,Black 3L
784,Tas Laptop Exsport Ladien Bana,https://www.tokopedia.com/exsportstore/tas-lap...,269000,269000,No Discount,22 terjual,5.0,https://images.tokopedia.net/img/cache/200-squ...,Olive
785,Tas Laptop Exsport Ladien Bana,https://www.tokopedia.com/exsportstore/tas-lap...,134500,134500,No Discount,6 terjual,5.0,https://images.tokopedia.net/img/cache/200-squ...,Navy
786,Tas Laptop Exsport Ladien Bana,https://www.tokopedia.com/exsportstore/tas-lap...,134500,134500,No Discount,17 terjual,5.0,https://images.tokopedia.net/img/cache/200-squ...,Maroon
787,Tas Laptop Exsport Ladien Bana,https://www.tokopedia.com/exsportstore/tas-lap...,134500,134500,No Discount,9 terjual,4.8,https://images.tokopedia.net/img/cache/200-squ...,Black
788,Tas Exsport Backpack Tauri,https://www.tokopedia.com/exsportstore/tas-exs...,275000,275000,No Discount,25 terjual,5.0,https://images.tokopedia.net/img/cache/200-squ...,Salem
789,Tas Exsport Taury Backpack,https://www.tokopedia.com/exsportstore/tas-exs...,137500,137500,No Discount,3 terjual,5.0,https://images.tokopedia.net/img/cache/200-squ...,Green
790,Tas Exsport Taury Backpack,https://www.tokopedia.com/exsportstore/tas-exs...,137500,137500,No Discount,6 terjual,5.0,https://images.tokopedia.net/img/cache/200-squ...,Gold
791,"Tas Selempang Exsport Dolfin Zarza,",https://www.tokopedia.com/exsportstore/tas-sel...,87600,87600,No Discount,12 terjual,5.0,https://images.tokopedia.net/img/cache/200-squ...,Grey 2L
